In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv
/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv


In [2]:

import pandas as pd 
import numpy as np 
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler, LabelEncoder 
from sklearn.linear_model import LogisticRegression 
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import StratifiedKFold, cross_val_score 
from sklearn.metrics import accuracy_score


train_df = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv") 
test_df = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv")

train_df.drop(columns=['Unnamed: 0'], inplace=True) 
test_df.drop(columns=['Unnamed: 0'], inplace=True)

X = train_df.drop(columns=['class', 'ID']) 
y = train_df['class'] 
X_test = test_df.drop(columns=['ID'])

for df in [X, X_test]: 
    df['mean'] = df.mean(axis=1) 
    df['std'] = df.std(axis=1) 
    df['min'] = df.min(axis=1) 
    df['max'] = df.max(axis=1) 
    df['median'] = df.median(axis=1) 
    df['skew'] = df.skew(axis=1)
    diffs = df.iloc[:, :-6].diff(axis=1).dropna(axis=1)
    df['diff_mean'] = diffs.mean(axis=1)
    df['diff_std'] = diffs.std(axis=1)

label_encoder = LabelEncoder() 
y_encoded = label_encoder.fit_transform(y)

pipeline = Pipeline([ ('imputer', SimpleImputer(strategy='mean')), ('scaler', StandardScaler()), ('classifier', LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs', C=0.5)) ])

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) 
scores = cross_val_score(pipeline, X, y_encoded, cv=skf, scoring='accuracy') 
print(f"Cross-validated Accuracy: {scores.mean():.5f} +/- {scores.std():.5f}")


pipeline.fit(X, y_encoded) 
test_preds_encoded = pipeline.predict(X_test) 
test_preds = label_encoder.inverse_transform(test_preds_encoded)

submission_df = pd.DataFrame({ 'ID': test_df['ID'], 'class': test_preds }) 
submission_df.to_csv("submission_logistic_regression.csv", index=False) 
print("Submission file saved as submission_logistic_regression.csv")



Cross-validated Accuracy: 0.91525 +/- 0.00432
Submission file saved as submission_logistic_regression.csv
